In [32]:
import os
import dotenv
import dspy
import json

In [33]:
dotenv.load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
proxy_url = os.getenv("PROXY_URL")
model = "gpt-4o-mini"

In [34]:
lm = dspy.LM(
    api_key=api_key,
    model=model,
    api_base=proxy_url,
    temperature=0.5,
)

dspy.configure(lm=lm)

In [37]:
class ClasificationSignature(dspy.Signature):
    """Classify if the input text is domain specific or not. If you are not sure about the domain classify as neutral."""

    prompt = dspy.InputField(desc="The input text to classify.")

    explanation = dspy.OutputField(desc="Reasoning behind the classification.")
    answer = dspy.OutputField(desc="Yes if the input text is domain specific, No otherwise. Neutral if not sure.")


class ClassificationSimple(dspy.Module):
    def __init__(self) -> None:
        super().__init__()

        self.prog = dspy.Predict(ClasificationSignature)
    
    def forward(self, prompt: str) -> ClasificationSignature:
        prediction = self.prog(prompt=prompt)
        return prediction

In [38]:
input = dspy.Example(prompt="I love cakes", explanation="The input text is about cakes.", answer="Yes").with_inputs("prompt")
output = ClassificationSimple()(input.prompt)

print(output)

Prediction(
    explanation='The statement "I love cakes" expresses a personal preference and does not refer to a specific domain or specialized knowledge. It is a general statement about a type of food rather than a technical or specialized subject.',
    answer='No'
)
